In [1]:
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import json

In [2]:
image_size = 224  # MobileNet typically uses 224x224 images

In [3]:
def create_class_mapping(dataset_dir):
    class_to_idx = {}
    current_idx = 0

    for folder in ['train', 'valid', 'test']:
        label_path = os.path.join(dataset_dir, folder, 'labels')
        
        for label_file in os.listdir(label_path):
            if label_file.endswith(".txt"):
                with (open(os.path.join(label_path, label_file), 'r')
                      as f):
                    lines = f.readlines()
                for line in lines:
                    class_name = line.split()[0]
                    if class_name not in class_to_idx:
                        class_to_idx[class_name] = current_idx
                        current_idx += 1
                        
    return class_to_idx

In [4]:
class CustomDataset(Dataset):
    def __init__(self, dataset_dir, folder, class_to_idx, 
                 transform=None):
        self.dataset_dir = dataset_dir
        self.folder = folder
        self.class_to_idx = class_to_idx
        self.transform = transform

        self.samples = []

        label_path = os.path.join(self.dataset_dir, self.folder, 
                                  'labels')
        image_path = os.path.join(self.dataset_dir, self.folder, 
                                  'images')

        for label_file in os.listdir(label_path):
            if label_file.endswith(".txt"):
                labels_and_bboxes = self.read_label_file(
                    os.path.join(label_path, label_file))
                image_file = os.path.join(image_path, label_file
                                          .replace('.txt', '.jpg'))
                
                for class_id, bbox in labels_and_bboxes:
                    self.samples.append((image_file, bbox, class_id))

    def read_label_file(self, label_file):
        with open(label_file, 'r') as f:
            lines = f.readlines()
        labels_and_bboxes = []
        for line in lines:
            label_data = line.strip().split()
            class_name = label_data[0]
            bbox = list(map(int, label_data[1:])) 
            class_id = self.class_to_idx[class_name]
            labels_and_bboxes.append((class_id, bbox))
        return labels_and_bboxes

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_file, bbox, label = self.samples[idx]
        
        image = cv2.imread(image_file)

        xmin, ymin, xmax, ymax = bbox
        cropped_image = image[ymin:ymax, xmin:xmax]

        if self.transform:
            cropped_image = self.transform(cropped_image)
        
        return cropped_image, label

In [5]:
# Load dataset
dataset_dir = "D:/Projects/ML/GCN/Dataset_classifier"
class_to_idx = create_class_mapping(dataset_dir)

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
print(class_to_idx)

{'LD': 0, 'HD': 1, 'other': 2, 'H': 3}


In [6]:
train_dataset = CustomDataset(dataset_dir, 'train', class_to_idx, 
                              transform=transform)
valid_dataset = CustomDataset(dataset_dir, 'valid', class_to_idx, 
                              transform=transform)
test_dataset = CustomDataset(dataset_dir, 'test', class_to_idx, 
                             transform=transform)

In [7]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Model load -- MobileNet_v3_small

In [8]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

In [9]:
num_classes = len(class_to_idx)

In [10]:
def build_model(num_classes):
    model = models.mobilenet_v3_small(
        weights='MobileNet_V3_Small_Weights.DEFAULT')
    model.classifier[3] = nn.Linear(
        model.classifier[3].in_features, num_classes)
    return model

model = build_model(num_classes)

# model = models.mobilenet_v3_small()
# model.classifier[3] = nn.Linear(
#     model.classifier[3].in_features,num_classes)
# model.load_state_dict(torch.load(
#     f"model_weights_v2/mobilenetv3_epoch_5.pth"))

<All keys matched successfully>

Model training

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [12]:
# Create a directory to save model weights
save_dir = 'model_weights_v2'
os.makedirs(save_dir, exist_ok=True)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [14]:
def train_model(model, train_loader, valid_loader, criterion, 
                optimizer, epochs=10):
    for epoch in range(5, epochs):
        model.train()
        running_loss = 0.0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
    
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.6f}")
        
        # Save model weights after each epoch
        torch.save(model.state_dict(), os.path.join(
            save_dir, f'mobilenetv3_epoch_{epoch+1}.pth'))
    
        # Validate
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
            
        val_accuracy = correct / total
        print(f"Validation Accuracy: {val_accuracy:.6f}")
        
    return model

In [ ]:
trained_model = train_model(model, train_loader, valid_loader, 
                            criterion, optimizer)

Epoch 6/20, Loss: 0.086517
Validation Accuracy: 0.952607
Epoch 7/20, Loss: 0.068452
Validation Accuracy: 0.948271
Epoch 8/20, Loss: 0.053509
Validation Accuracy: 0.948271
Epoch 9/20, Loss: 0.040926
Validation Accuracy: 0.947135
Epoch 10/20, Loss: 0.034661
Validation Accuracy: 0.947858


Model validation

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v3_small()
model.classifier[3] = nn.Linear(model.classifier[3].in_features,
                                num_classes)

for i in range(1, 6):
    # Load model state
    state_dict = torch.load(f"model_weights/mobilenetv3_epoch_{i}.pth")

    # Load it into model
    model.load_state_dict(state_dict)

    model = model.to(device)

    # Validate
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
        
    val_accuracy = correct / total
    print(f"Validation Accuracy after epoch {i}: {val_accuracy:.4f}")

Validation Accuracy after epoch 1: 0.9313
Validation Accuracy after epoch 2: 0.9417
Validation Accuracy after epoch 3: 0.9455
Validation Accuracy after epoch 4: 0.9513
Validation Accuracy after epoch 5: 0.9402


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v3_small()
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 
                                num_classes)

criterion = nn.CrossEntropyLoss()

for i in range(1, 6):
    # Load model state
    state_dict = torch.load(f"model_weights/mobilenetv3_epoch_{i}.pth")

    # Load it into model
    model.load_state_dict(state_dict)

    model = model.to(device)

    # Validate
    model.eval()
    running_loss = 0.0
    
    for inputs, labels in valid_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(valid_loader.dataset)
    print(f"Loss {i}: {epoch_loss:.6f}")

Loss 1: 0.160926
Loss 2: 0.144894
Loss 3: 0.130532
Loss 4: 0.131385
Loss 5: 0.150494
